# Deep Neural Network for MNIST Classification
We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs).

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image).

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes.

Our goal would be to build a neural network with 2 hidden layers

## Import the relevant libraries

In [1]:
import numpy as np
import tensorflow as tf

# tensorflow has a dataset that needs to be installed separately, so if you dont have it
# you can install it using anaconda, or pip install tensorflow-datasets or conda install tensorflow-datasets
import tensorflow_datasets as tfds
# the first time you download a dataset using this library it will be stored in 
# C:\Users\*USERNAME*\tensorflow_datasets\... and the next time it uses one which is in your computer

## Load the Data

In [2]:
# mnist_dataset, mnist_info = tfds.load(name= 'mnist', with_info= True, as_supervised= True) # to find the exact name of the dataset refer to 
# https://www.tensorflow.org/datasets/catalog/overview
mnist_dataset, mnist_info = tfds.load(name= 'mnist', with_info= True, as_supervised= True)

# as_supervised = True: this will load the data in a 2-tuple structure [input, target]
# alternatively, as_supervised=False, would return a dictionary
# obviously we prefer to have our inputs and targets separated 

# mnist_info : ... with_info = True : provides a tuple containing information about version, features, 
# and number of samples of dataset . we will use this information a bit below and we will store it in mnist_info

# the first time it takes a bit longer because you are actually downloading the data set, but next time 
# it will not take time. 

## Data preprocessing

### 1- Define the size of the data for each three subdata : training, validation, and test  
Here before splitting we need to define the number(size) of each sample and then do standarization and shuffling and finally split hte data

In [3]:
# as the validation data is a part of train data, first we split it into two subset: train and test
# once we have loaded the dataset, we can easily extract the training and testing dataset with the built references
# here luckily our dataset has split it already
# if you go to the tesnorflow website, in statistics part it is mentioned the 60000 train and 10000 test
# but no validation data 
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# by default, TF has training and testing datasets, but no validation sets
# thus we must split it on our own

# we start by defining the number of validation samples as a % of the train samples
# this is also where we make use of mnist_info (we don't have to count the observations)
# the percent we chose if 10% = 0.1
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

# as 0.1 * number of train data set, might not be an integer we should fix to make sure we get an int number
# and it is like round function
# let's cast(convert) this number to an integer, as a float may cause an error along the way
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# let's also store the number of test samples in a dedicated variable (instead of using the mnist_info one)
num_test_samples = mnist_info.splits['test'].num_examples

# once more, we'd prefer an integer (rather than the default float)
num_test_samples = tf.cast(num_test_samples, tf.int64)

print(mnist_info.splits['train'].num_examples)
print(num_validation_samples)
print(num_test_samples)

60000
tf.Tensor(6000, shape=(), dtype=int64)
tf.Tensor(10000, shape=(), dtype=int64)


### 2- scaling (standardizing) input

#### Define a scaling function that gets input and label and returns scaled_input and label

In [4]:
# normally, we would like to scale our data in some way to make the result more numerically stable
# in this case we will simply prefer to have inputs between 0 and 1 
# note that we do not want to standardize data with mean 0 and std =1, only between 0 and 1
# let's define a function called: scale, that will take an MNIST image (as matrix whose elements are 
# between 0 and 255) and its label and return image whose matrix's elements are scaled between 0 and 1 and label

def scale(image, label):
    # we make sure the value is a float
    image = tf.cast(image, tf.float32)
    # since the possible values for the inputs are 0 to 255 (256 different shades of grey)
    # if we divide each element by 255, we would get the desired result -> all elements will be between 0 and 1 
    scaled_image = image/255.  # dont forget "." at the end of 255, since we want the image_scaled be a matrix with float numbers

    return scaled_image, label


#### Apply our scaling function to our dataset

In [5]:
# the method .map() allows us to apply a custom transformation to a given dataset
# we have already decided that we will get the validation data from mnist_train, so 
# this will scale the whole train dataset and store it in a new variable
scaled_train_and_validation_data = mnist_train.map(scale)

# finally, we scale and batch the test data
# we scale it so it has the same magnitude as the train and validation
# there is no need to shuffle it, because we won't be training on the test data
# there would be a single batch, equal to the size of the test data
scaled_test_data = mnist_test.map(scale)
scaled_train_and_validation_data

<MapDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

### 3- Shuffle the train (train and validation) data

In [6]:
# shuffling = keeping the same information but in a different order
# beacuse most of the data sets are likely to be ordered, so if we batch them in each bach
# we have the same for example target, which distracts the learning process. 
# let's shuffle the data

BUFFER_SIZE = 10000
# this BUFFER_SIZE parameter is here for cases when we're dealing with enormous datasets
# then we can't shuffle the whole dataset in one go because we can't fit it all in memory
# so instead TF only stores BUFFER_SIZE samples in memory at a time and shuffles them
# if BUFFER_SIZE=1 => no shuffling will actually happen
# if BUFFER_SIZE >= num samples => shuffling is uniform and only once : if buffersize >=60000
# 1<BUFFER_SIZE<60000 we will be optimizing the computational power approximate uniform shuffling

# luckily for us, there is a shuffle method readily available and we just need to specify the buffer size
shuffled_scaled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# once we have scaled and shuffled the data, we can proceed to actually extracting the train and validation
# our validation data would be equal to 10% of the training set, whose number have already been calculated and stored in num_validation_samples
# we use the .take() method to take(extract) that many samples
# finally, we create a batch with a batch size equal to the total number of validation samples
shuffled_scaled_validation_data = shuffled_scaled_train_and_validation_data.take(num_validation_samples)

# similarly, the train_data is everything else, so we skip as many samples as there are in the validation dataset
# it gets all the data except validation data
shuffeled_scaled_train_data = shuffled_scaled_train_and_validation_data.skip(num_validation_samples)


### 4- batching data and change the structure of the validation data

In [8]:
# determine the batch size
BATCH_SIZE = 100
# we have 60000 training and validation, 10% for validation, so 54000 only for training, batch size =100, so
# 540000/100 = 540 number of out batches, at the end of each batch, we get one weight vector and one bias vector. 

# we can also take advantage of the occasion to batch the train data
# this would be very helpful when we train, as we would be able to iterate over the different batches
# dataset.batch(batch_size) is a method that combines the consecuitive elements of a dataset inti batches
# remember that batching was useful in updating weights only once per batch,which is like 100 samples, so
# rather than at every sample, so when batching we usually find the average loss and average accuracy. 
train_data = shuffeled_scaled_train_data.batch(BATCH_SIZE)

# so during training we use the average loss and average accuracy, but for validation and test we need the 
# exact loss and accuracy. 
# as validation data is used only in forward propagation, we don't really need to batch, but the model
# expects us to have all the data in batvh form, so we define : batch size = sample size
# to take all the data at once. 
validation_data = shuffled_scaled_validation_data.batch(num_validation_samples)

# similarily batch the test data
test_data = scaled_test_data.batch(num_test_samples)


# takes next batch (it is the only batch)
# our validatiob data must have the same shape and object properties as the train and test data
# because as_supervized=True, we've got an 2-tuple structure, also the mnist data is iterable
# so we must extract and convert the validation input and target appropriately. 
# iter() : creates an object which can be iterated one element at a time to make iterable to be used in for or while loop
# next: loads the next batch, since there is only one batch it will load the input and the targets. 
validation_inputs, validation_targets = next(iter(validation_data))

## Build the model 

### 1- Outline the model

In [14]:
input_size = 784  # 28*28 => 1*784
output_size = 10  # 0,1,2,3,4,5,6,7,8,9

# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 100
    
# define how the model will look like
model = tf.keras.Sequential([
    
    # the first layer (the input layer)
    # each observation is 28x28x1 pixels, therefore it is a tensor of rank 3
    # since we don't know CNNs yet, we don't know how to feed such input into our net, so we must flatten the images into a vector
    # so we use flatten method
    # this convenient method 'Flatten' that simply takes our 28x28x1 tensor and orders it into a (None,) 
    # or (28x28x1,) = (784,) vector
    # this allows us to actually create a feed forward neural network
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer
    
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    # we use 'relu' as our activation functions
    # to know which activation function is better we can simply run the model with different activation functions and compare the results
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    
    # the final layer is no different, we just make sure to activate it with softmax
    # why softmex: when we are creating a classifier, the activation function of the output layer must transform values into probabilities.
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

### 2- choose the optimizer and the loss function

In [15]:
# we define the optimizer we'd like to use, we know that ADAM is the best becuase it combines momentum with RMSprop 
# the loss function, cross entropthy is the first option, however there are different types of cross entrophy. in tensorflow2
# there are 3 differnet built-in variations of cross entrophy: binary cross entrophy, categorical cross entrophy, and sparse categorical cross entrophy.   
# binary cross entrophy is for binary encoding, 
# categorical cross entrophy: for one-hot encoding,
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### 3- training

In [16]:
# determine the maximum number of epochs
NUM_EPOCHS = 5

# we fit the model, specifying the
# training data
# the total number of epochs
# and the validation data we just created ourselves in the format: (inputs,targets)
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), validation_steps=1, verbose =2)

Epoch 1/5
540/540 - 7s - loss: 0.3292 - accuracy: 0.9054 - val_loss: 0.1715 - val_accuracy: 0.9540
Epoch 2/5
540/540 - 6s - loss: 0.1369 - accuracy: 0.9598 - val_loss: 0.1198 - val_accuracy: 0.9658
Epoch 3/5
540/540 - 7s - loss: 0.0951 - accuracy: 0.9720 - val_loss: 0.0992 - val_accuracy: 0.9723
Epoch 4/5
540/540 - 7s - loss: 0.0744 - accuracy: 0.9776 - val_loss: 0.0833 - val_accuracy: 0.9770
Epoch 5/5
540/540 - 7s - loss: 0.0589 - accuracy: 0.9815 - val_loss: 0.0724 - val_accuracy: 0.9790


In [13]:
# 540/540 : number of batches
# loss: training loss, compare across epochs, it is decreasing. 
# accuracy: shows in what percent of the cases our output were equal to the targets, increasing
# val_loss: validation loss, we keep an eye on that to determine whether the model is overfitting.
# val_accuracy: validation accuracy: is the true accuracy of the model at the end of each epoch
# accuracy: train accuracy is the accuracy across batches while val accuracy is that of the whole validation set. 
# validation accuracy of the last epoch = 0.9695 : which is a remarkable result. 
# by changing hyper parmaneters we might get better results. such as hidden layer size 50 to 100 : 96.95% to 97.90%

### What heppens inside an epoch:
1- at the beginning of each epoch, the training loss will be set to 0

2- The algorithm will iterate over a present number of batches, all from train_data (essentially the whole train data will be utilized but in batches, therefore

3- The weights and biases will be updated as many times as there are batches.

4- we will get a value for the loss function, indicating how the training is going.

5- we also see the training accuracy. 

6- at the end of the epoch, the algorithm will forward propagate the whole validation set through the optimized model and calculate the validation accuracy. 

7- When we reach the maximum number of epoches (5) the training will be over. 

### 4- Test the model
by fiddeling with the hyper parameters in the above procedure, we can imporve validation accuracy and may get the best combination of hyper parameters. but the thing here is that those hyper parameters we have found are not the best general hyper parameters, but rather the best for our validation data set, which fit our validation data best. meaning that by finding them we are overfitting our validation data set. 
so, as validation check helps us avoid overfitting train data set, test check helps us avoid overfitting hyper parameters for validation data. 

In [18]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 1s 1s/step - loss: 0.0806 - accuracy: 0.9754

In [19]:
# We can apply some nice formatting if we want to
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.54%


In [ ]:
# after we test the model, conceptually, we are no longer allowed to change it. 
# becuase our model has seen the data set. so we can do it only once. 
# getting an accuracy close to validation accuracy is or model did not overfit. 
# test accuracy is what we expect to get for real world data. 